In [1]:
import requests
import numpy as np
from scipy.stats import norm
import timeseries as ts

In [2]:
# get
requests.get("http://localhost:5000/api/timeseries").json()

{'Payload': [], 'Status': 'OK'}

In [3]:
# taken from go_client.py
# insert via rest api
def tsmaker(m, s, j):
    "returns metadata and a time series in the shape of a jittered normal"
    meta={}
    meta['order'] = int(np.random.choice([-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]))
    meta['blarg'] = int(np.random.choice([1, 2]))
    t = np.arange(0.0, 1.0, 0.01)
    v = norm.pdf(t, m, s) + j*np.random.randn(100)
    return meta, ts.TimeSeries(t, v)

mus = np.random.uniform(low=0.0, high=1.0, size=50)
sigs = np.random.uniform(low=0.05, high=0.4, size=50)
jits = np.random.uniform(low=0.05, high=0.2, size=50)
for i, m, s, j in zip(range(50), mus, sigs, jits):
    meta, tsrs = tsmaker(m, s, j)
    # the primary key format is ts-1, ts-2, etc
    pk = "ts-{}".format(i)
    meta['vp'] = False # augment metadata with a boolean asking if this is a  VP.
    meta['pk'] = pk
    requests.post("http://localhost:5000/api/timeseries", json={"pk":pk, "t":list(tsrs.times()), "v":list(tsrs.values())})
    requests.post("http://localhost:5000/api/timeseries/upsert", json=meta)

In [4]:
# get ts-0
requests.get("http://localhost:5000/api/timeseries?field1=pk&value1=ts-0").json()

{'Payload': [['ts-0', {'blarg': 1, 'order': 4, 'pk': 'ts-0', 'vp': False}]],
 'Status': 'OK'}

In [5]:
# commit to disk
requests.post("http://localhost:5000/api/commit").json()

{'Payload': None, 'Status': 'OK'}

In [6]:
# delete ts
requests.delete("http://localhost:5000/api/timeseries?pk=ts-0").json()

{'Payload': None, 'Status': 'OK'}

In [7]:
# see that it's gone
requests.get("http://localhost:5000/api/timeseries?field1=pk&value1=ts-0").json()

{'Payload': [], 'Status': 'OK'}

In [8]:
# oops, want to rollback
requests.post("http://localhost:5000/api/rollback").json()

{'Payload': None, 'Status': 'OK'}

In [9]:
# get back ts
requests.get("http://localhost:5000/api/timeseries?field1=pk&value1=ts-0").json()

{'Payload': [['ts-0', {'blarg': 1, 'order': 4, 'pk': 'ts-0', 'vp': False}]],
 'Status': 'OK'}

In [10]:
# get mean of timeseries in decreasing order
requests.get("http://localhost:5000/api/timeseries/augmented?proc=stats&target=mean&target=std&sort_by=mean&sort_by_increasing=false").json()

{'Payload': [['ts-0', {'mean': 1.0126144909822037, 'std': 1.1121798459260237}],
  ['ts-1', {'mean': 1.009356191928828, 'std': 1.820963904271178}],
  ['ts-10', {'mean': 1.0067317047470374, 'std': 1.5831466334251727}],
  ['ts-11', {'mean': 1.0006644988977624, 'std': 1.2758834081658386}],
  ['ts-12', {'mean': 0.993129876641768, 'std': 1.9511960338676577}],
  ['ts-13', {'mean': 0.9892923408907304, 'std': 0.9289356857562812}],
  ['ts-14', {'mean': 0.9887912674789371, 'std': 0.8900866532200021}],
  ['ts-15', {'mean': 0.9835486015632569, 'std': 1.0931256747941671}],
  ['ts-16', {'mean': 0.9792311626020498, 'std': 1.5670267047496198}],
  ['ts-17', {'mean': 0.9760702178357612, 'std': 0.9769949230005534}],
  ['ts-18', {'mean': 0.9646987868749026, 'std': 1.3547280455620658}],
  ['ts-19', {'mean': 0.9632287919664444, 'std': 0.5088563357523552}],
  ['ts-2', {'mean': 0.9527951983960402, 'std': 0.8516322894133211}],
  ['ts-20', {'mean': 0.9487062916186098, 'std': 0.669304568104767}],
  ['ts-21', {'me

In [11]:
# get similarity specifying five timeseries as vantage points
requests.get("http://localhost:5000/api/timeseries/similarity?pk1=ts-18&pk2=ts-10&pk3=ts-23&pk4=ts-32&pk5=ts-15&sort_by=d_vp-1").json()

{'Payload': [['ts-18',
   {'d_vp-1': 0.0,
    'd_vp-2': 1.4141665932765386,
    'd_vp-3': 1.4130253087403384,
    'd_vp-4': 1.4140495543154987}],
  ['ts-19',
   {'d_vp-1': 1.155479606144914,
    'd_vp-2': 1.4142124668453937,
    'd_vp-3': 1.4138972191867392,
    'd_vp-4': 1.414075539386752}],
  ['ts-41',
   {'d_vp-1': 1.2514583225641192,
    'd_vp-2': 1.4138895446973747,
    'd_vp-3': 1.411053102782802,
    'd_vp-4': 1.4134999777822574}],
  ['ts-43',
   {'d_vp-1': 1.3050622993922183,
    'd_vp-2': 1.4118524052958692,
    'd_vp-3': 1.4106486386507386,
    'd_vp-4': 1.4141721445381412}],
  ['ts-29',
   {'d_vp-1': 1.3726722855824858,
    'd_vp-2': 1.4134751430294623,
    'd_vp-3': 1.4110668858996902,
    'd_vp-4': 1.4141533894378098}],
  ['ts-1',
   {'d_vp-1': 1.3914232906770092,
    'd_vp-2': 1.4142127103646858,
    'd_vp-3': 1.4081425724479986,
    'd_vp-4': 1.4067319614715414}],
  ['ts-42',
   {'d_vp-1': 1.3935226998030505,
    'd_vp-2': 1.372898500773963,
    'd_vp-3': 1.4141580248457

In [12]:
# delete all timeseries and commit
for i in range(50):
    pk = "ts-{}".format(i)
    requests.delete("http://localhost:5000/api/timeseries?pk={}".format(pk))
requests.post("http://localhost:5000/api/commit").json()

{'Payload': None, 'Status': 'OK'}

In [13]:
# get all
requests.get("http://localhost:5000/api/timeseries").json()

{'Payload': [], 'Status': 'OK'}